### 문제 5-1 : 카페 메뉴 도구(Tool) 호출 체인 구현




In [6]:

from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_community.tools import TavilySearchResults
from langchain_core.tools import tool
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
from langchain.document_loaders import TextLoader
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableConfig, chain
import os
from langchain_community.document_loaders import WikipediaLoader
from langchain_core.runnables import RunnableLambda
from pydantic import BaseModel, Field
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage

load_dotenv()

True

In [2]:

# vector db 설정
loader = TextLoader("./data/cafe_menu_data.txt", encoding="utf-8")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
texts = text_splitter.split_documents(loader.load())


# llm 설정
llm = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1", 
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

# embed 설정
embeddings = OllamaEmbeddings(model="bge-m3")

# db 설정
db_path = os.path.join("./db", "cafe_db")
db = FAISS.from_documents(texts, embeddings)
db.save_local(db_path)

loaded_db = FAISS.load_local(db_path, embeddings, allow_dangerous_deserialization=True)

    

C:\Users\dasua\AppData\Local\Temp\ipykernel_16164\1470150652.py:15: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model="bge-m3")


In [9]:
# 도구 정의

class TavilySearchInput(BaseModel):
    query: str = Field(description="검색할 쿼리 문자열")

class WikipediaInput(BaseModel):
    query: str = Field(description="위키백과에서 검색할 주제")

class CafeDBSearchInput(BaseModel):
    query: str = Field(description="로컬 카페 메뉴 DB에서 검색할 메뉴 관련 쿼리")


tavily_search_tool_internal = TavilySearchResults()
@tool(args_schema=TavilySearchInput)
def tavily_search_func(query: str) -> str:
    """웹에서 최신 정보를 검색합니다."""
    return tavily_search_tool_internal.invoke({"query": query})


@tool(args_schema=WikipediaInput)
def wiki_summary(query: str) -> str:
    """위키피디아에서 일반 지식을 검색하고 요약합니다."""
    try:
        # WikipediaLoader를 사용하여 문서 로드
        loader = WikipediaLoader(query=query, load_max_docs=1, doc_content_chars_max=500)
        docs = loader.load()
        if docs:
            return docs[0].page_content
        else:
            return "위키피디아에서 해당 정보를 찾을 수 없습니다."
    except Exception as e:
        return f"위키피디아 검색 중 오류 발생: {e}"


@tool(args_schema=CafeDBSearchInput)
def db_search_cafe_func(query: str) -> str:
    """
    로컬 카페 메뉴 DB에서 정보(가격, 재료, 설명)를 검색합니다.
    사용자 질문에서 메뉴 이름을 정확히 추출하여 호출해야 합니다.
    예: "아메리카노 가격" -> query="아메리카노"
    """
    docs = loaded_db.similarity_search(query)
    if not docs:
        return "해당 메뉴 정보를 찾을 수 없습니다."
    
    return "\n---\n".join([doc.page_content for doc in docs])

# LLM에 도구 바인딩
tools = [tavily_search_func, wiki_summary, db_search_cafe_func]
llm_with_tools = llm.bind_tools(tools)



@chain
def cafe_search_chain(user_input: str, config: RunnableConfig):
    ai_msg = llm_with_tools.invoke([HumanMessage(content=user_input)], config=config)
    
    if not ai_msg.tool_calls:
        print("도구 호출 없이 일반 응답 생성.")
        return ai_msg.content if ai_msg.content else "질문에 답변할 수 없습니다."

    tool_results_list = []
    
    for tool_call in ai_msg.tool_calls:
        tool_name = tool_call["name"]
        tool_args = tool_call["args"]
        
        print(f"\n--- 도구 호출: {tool_name} ---")
        print(f"인자: {tool_args}")
        print("-" * 100)

        tool_output_content = None
        try:
            if tool_name == "tavily_search_func":
                tool_output_content = tavily_search_func.invoke(tool_args.get("query", ""), config=config)
            elif tool_name == "wiki_summary":
                tool_output_content = wiki_summary.invoke(tool_args.get("query", ""), config=config)
            elif tool_name == "db_search_cafe_func":
                tool_output_content = db_search_cafe_func.invoke(tool_args.get("query", ""), config=config)
            else:
                tool_output_content = f"알 수 없는 도구: {tool_name}"
        except Exception as e:
            tool_output_content = f"도구 실행 중 오류 발생 ({tool_name}): {e}"
        
        print(f"도구 {tool_name} 결과 (일부): {tool_output_content[:200]}...") 
        tool_results_list.append(f"<{tool_name} 결과>\n{tool_output_content}")
    
    
    cafe_prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a helpful cafe assistant. Provide accurate information based on the search results."),
        ("human", "{user_input}"), 
        AIMessage(content=ai_msg.content if ai_msg.content else "도구를 사용하여 정보를 검색했습니다."), 
        ("human", "검색 결과: {tool_results}")
    ])
    
    tool_results_str = "\n\n".join(tool_results_list)
    
    cafe_chain = cafe_prompt | llm
    
    final_response = cafe_chain.invoke({
        "user_input": user_input,
        "tool_results": tool_results_str
    }, config=config) 
    
    return final_response.content


query = "라떼들의 평균 비용은 얼마임?"
response = cafe_search_chain.invoke(query)
print("질문: ", query)
print(response)


--- 도구 호출: db_search_cafe_func ---
인자: {'query': '안력 라딌베 비용'}
----------------------------------------------------------------------------------------------------
도구 db_search_cafe_func 결과 (일부): 4. 바닐라 라떼
   • 가격: ₩6,000
   • 주요 원료: 에스프레소, 스팀 밀크, 바닐라 시럽
   • 설명: 카페라떼에 달콤한 바닐라 시럽을 더한 인기 메뉴입니다. 바닐라의 달콤함과 커피의 쌉싸름함이 조화롭게 어우러지며, 휘핑크림 토핑으로 더욱 풍성한 맛을 즐길 수 있습니다.

5. 카라멜 마키아토
   • 가격: ₩6,500
   • 주요 원...

--- 도구 호출: db_search_cafe_func ---
인자: {'query': '베리스탈 라딌베 비용'}
----------------------------------------------------------------------------------------------------
도구 db_search_cafe_func 결과 (일부): 4. 바닐라 라떼
   • 가격: ₩6,000
   • 주요 원료: 에스프레소, 스팀 밀크, 바닐라 시럽
   • 설명: 카페라떼에 달콤한 바닐라 시럽을 더한 인기 메뉴입니다. 바닐라의 달콤함과 커피의 쌉싸름함이 조화롭게 어우러지며, 휘핑크림 토핑으로 더욱 풍성한 맛을 즐길 수 있습니다.

5. 카라멜 마키아토
   • 가격: ₩6,500
   • 주요 원...

--- 도구 호출: db_search_cafe_func ---
인자: {'query': '마릴라틈 비용'}
----------------------------------------------------------------------------------------------------
도구 db_search_cafe_func 결과 (일부): 4. 바